In [1]:
%%time

import tensorflow as tf
print(tf.__version__)
import sys
sys.path.append("..")
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np


def plot_loss(history, *losses):
    for loss in losses:
        plt.plot(history.history[loss], label=loss)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

def scaling(x, min, max):
    return np.where(x < min, 0.0, np.where(x > max, 1.0, (x - min) / (max - min)))

early_stopping = EarlyStopping(
    monitor='val_loss',  # 
    patience=50,        # 
    verbose=1,          # 
    mode='min',         # 
    restore_best_weights=True  # 
)


from keras.callbacks import Callback

class TerminateOnNegative(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        loss = logs.get('loss')
        if loss is not None:
            if loss < -0.5:
                print('Negative loss detected and terminating training')
                self.model.stop_training = True

2024-02-20 15:35:10.141458: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-20 15:35:10.176843: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 15:35:10.176867: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 15:35:10.176889: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-20 15:35:10.183627: I tensorflow/core/platform/cpu_feature_g

2.14.0
CPU times: user 2.6 s, sys: 3.42 s, total: 6.02 s
Wall time: 2 s


In [2]:
SAVE_DIR = "../data"
file_criteo = SAVE_DIR + "/MT-LIFT/train.csv"
df_criteo_ori = pd.read_csv(file_criteo, sep=',')

In [24]:
pd.set_option('display.max_columns', None)
df_criteo_ori.describe()

,click,conversion,treatment,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98
count,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4563026.0,4563026.0,4563026.0,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06,4.563026e+06
mean,3.329111e-01,6.808859e-02,2.017062e+00,1.297720e-14,-4.963344e-14,1.297720e-14,1.858299e-15,1.132534e+01,4.847851e+01,1.226735e-14,4.893479e-14,2.055761e-01,-7.029400e-15,-1.531296e-14,-9.767474e-17,-1.433504e-15,-7.158479e-16,1.729946e-15,-1.000455e-16,5.403623e-16,1.206037e-15,3.792545e-16,5.069362e-17,8.647477e-16,-2.808954e-15,1.551954e-15,1.049387e-15,-1.744694e-15,1.477405e-15,3.019315e-16,1.888722e-14,-2.648952e-15,-1.580736e-15,-4.773777e-16,2.719365e-14,-1.152346e-15,1.982184e-15,2.989136e-15,-4.611154e-14,1.141461e-13,5.541246e-14,-1.341142e-13,-4.068228e-15,1.854724e-15,2.773904e-15,1.543437e-15,-6.015715e-15,3.699764e-15,-8.055069e-15,3.585170e-14,2.412674e-14,-2.079914e-15,1.707669e-14,-1.223745e-14,1.835816e-14,-1.023446e-14,3.965333e-15,6.341540e-14,2.078108e-14,1.140032e-13,4.469076e-14,5.683151e-15,-1.386516e-14,-1.821274e-13,-1.448995e-13,8.104543e-15,-3.443619e-13,1.545818e-13,-4.948753e-14,2.046312e-14,8.860513e-14,1.135963e-14,3.641478e-15,-3.217927e-14,3.388089e-14,-8.494551e-15,2.398633e-14,-5.253318e-14,-6.736687e-15,1.916000e-13,-1.341227e-13,-1.148650e-13,9.186658e-13,0.0,0.0,0.0,-2.054864e-13,-1.436688e-13,7.997414e-14,3.189143e-14,3.382649e-14,4.331797e-16,-8.919827e-14,-3.326532e-13,-1.550075e-13,-2.684486e-13,4.524844e-14,-5.131995e-15,-3.978028e-14,-2.948881e-15,3.773022e-14,-7.525065e-16
std,4.712551e-01,2.518979e-01,1.476708e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.345643e+00,3.750672e+01,1.000000e+00,1.000000e+00,7.129533e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.0

In [7]:
value_counts = df_criteo_ori['treatment'].value_counts()

print(value_counts)

3    1646545
0    1269914
4     699450
2     519355
1     427762
Name: treatment, dtype: int64


In [3]:
%%time

sample = 1.0
random_state=20220720
df_criteo=df_criteo_ori[(df_criteo_ori['treatment'] == 0) | (df_criteo_ori['treatment'] == 3)].sample(frac=sample, random_state=random_state).reset_index(drop=True)
# Change 'treatment' from 3 to 1 in df_criteo_ori
df_criteo['treatment'] = df_criteo['treatment'].replace(3, 1)

# X = df_criteo[['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11']].values

# X[:, 0] = scaling(X[:, 0], min=np.min(X[:, 0]), max=np.max(X[:, 0]))
# X[:, 1] = scaling(X[:, 1], min=np.min(X[:, 1]), max=np.max(X[:, 1]))
# X[:, 2] = scaling(X[:, 2], min=np.min(X[:, 2]), max=np.max(X[:, 2]))
# X[:, 3] = scaling(X[:, 3], min=np.min(X[:, 3]), max=np.max(X[:, 3]))
# X[:, 4] = scaling(X[:, 4], min=np.min(X[:, 4]), max=np.max(X[:, 4]))
# X[:, 5] = scaling(X[:, 5], min=np.min(X[:, 5]), max=np.max(X[:, 5]))
# X[:, 6] = scaling(X[:, 6], min=np.min(X[:, 6]), max=np.max(X[:, 6]))
# X[:, 7] = scaling(X[:, 7], min=np.min(X[:, 7]), max=np.max(X[:, 7]))
# X[:, 8] = scaling(X[:, 8], min=np.min(X[:, 8]), max=np.max(X[:, 8]))
# X[:, 9] = scaling(X[:, 9], min=np.min(X[:, 9]), max=np.max(X[:, 9]))
# X[:, 10] = scaling(X[:, 10], min=np.min(X[:, 10]), max=np.max(X[:, 10]))
# X[:, 11] = scaling(X[:, 11], min=np.min(X[:, 11]), max=np.max(X[:, 11]))
# # 
columns = [f'f{i}' for i in range(99) if not (80 <= i <= 82)] 
X = df_criteo[columns].values
# 
for i in range(X.shape[1]):
    # 
    if i not in [80, 81, 82]:
        X[:, i] = scaling(X[:, i], min=np.min(X[:, i]), max=np.max(X[:, i]))

T = df_criteo['treatment'].values.reshape(-1, 1)
Y_visit = df_criteo['click'].values.reshape(-1, 1)
Y_conv = df_criteo['conversion'].values.reshape(-1, 1)

T.shape, Y_visit.shape, Y_conv.shape


# calculate len
train_len = int(len(X) * 0.70)
cali_len = int(len(X) * 0.05)
test_len = len(X) - train_len - cali_len

# obtain train set
X_train = X[:train_len, :]
T_train = T[:train_len, :]
Y_visit_train = Y_visit[:train_len, :]
Y_conv_train = Y_conv[:train_len, :]

# obtain calibration set
X_cali = X[train_len:train_len+cali_len, :]
T_cali = T[train_len:train_len+cali_len, :]
Y_visit_cali = Y_visit[train_len:train_len+cali_len, :]
Y_conv_cali = Y_conv[train_len:train_len+cali_len, :]

# obtain test set
X_test = X[train_len+cali_len:, :]
T_test = T[train_len+cali_len:, :]
Y_visit_test = Y_visit[train_len+cali_len:, :]
Y_conv_test = Y_conv[train_len+cali_len:, :]

print(train_len, X_train.shape, X_test.shape, len(X), X_cali.shape)

# make covariate shift
# condition_cali = (X_cali[:, 0] > 0.3) & (X_cali[:, 1] < 0.7)
# X_cali = X_cali[condition_cali]
# T_cali = T_cali[condition_cali]
# Y_visit_cali = Y_visit_cali[condition_cali]
# Y_conv_cali = Y_conv_cali[condition_cali]

# condition_test = (X_test[:, 0] > 0.3) & (X_test[:, 1] < 0.7)
# X_test = X_test[condition_test]
# T_test = T_test[condition_test]
# Y_visit_test = Y_visit_test[condition_test]
# Y_conv_test = Y_conv_test[condition_test]

# print(train_len, X_train.shape, X_test.shape, len(X), X_cali.shape)

2041521 (2041521, 96) (729116, 96) 2916459 (145822, 96)
CPU times: user 4.67 s, sys: 3.54 s, total: 8.21 s
Wall time: 8.2 s


# First, on the train set:

### (i) Train DRP model

In [ ]:
%%time
# DPR on train set
sys.path.append("..")
from model.uplift_model import *

count = 1
# 2.5e-5
import keras
import keras.backend as K
import tensorflow as tf
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from model.roi_model import *

# final_model = get_roi_rank_criteo_model()
# final_model.compile(loss=None, optimizer='adam')
#lambda y_true,y_pred: y_pred
# print('trainable_weights')
# for x in final_model.trainable_weights:
#     print(x.name)
# print('non_trainable_weights')
# for x in final_model.non_trainable_weights:
#     print(x.name)
# final_model.summary()

for i in range(count):

    print("iteration = ", i + 1)
    
    final_model = get_roi_rank_model()
    final_model.compile(loss=None, optimizer='adam')

    mcp_save = ModelCheckpoint('../model_file/roi/criteo/final_model/roi_rank/roi_rank_criteo_model_{}_{}.h5'.format(i+1, sample), save_best_only=False, monitor='val_loss', mode='min', save_weights_only=True)
    history = final_model.fit([X_train, T_train, Y_conv_train, Y_visit_train], validation_split=0.2, epochs=1000, batch_size=2000000, shuffle=True, verbose=1, callbacks=[mcp_save,TerminateOnNegative()])
    plot_loss(history, "loss", "val_loss", "obj", "val_obj")

# Second, on the calibration set:

### (i) Infer DRP model to obtain $\hat{roi}$.

In [4]:
%%time
import sklearn 
import sklearn.metrics
from metric.Metric import *
sys.path.append("..")
from model.roi_model import *

count = 1

DRP_aucc_cali_list = []
roi_rank_pre_cali_list = []
for i in range(count):
    
    print("iteration = ", i + 1)
    
    final_model = get_roi_rank_model()
    final_model.load_weights('../model_file/roi/criteo/final_model/roi_rank/roi_rank_criteo_model_{}_{}.h5'.format(i+1, sample))

    print("cali")
    roi_rank_pre_cali = final_model.predict([X_cali, T_cali, Y_conv_cali, Y_visit_cali])
    DRP_aucc = get_uplift_model_aucc_no_show(t=(T_cali > 0.5).flatten(), y_reward=Y_conv_cali.flatten(), y_cost=Y_visit_cali.flatten(), roi_pred=roi_rank_pre_cali.flatten(), quantile=200)
    roi_rank_pre_cali_list.append(roi_rank_pre_cali)
    DRP_aucc_cali_list.append(DRP_aucc)
    
    print("\n")

iteration =  1


2024-02-20 15:38:33.150450: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22288 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:1f:00.0, compute capability: 8.9


cali
4557/4557 [==============================] - 10s 2ms/step
AUCC =  0.7202660262850933


CPU times: user 17 s, sys: 1.16 s, total: 18.1 s
Wall time: 12.9 s


### (ii) Calculate $roi^*$ by Algorithm 2.

In [5]:
import numpy as np
from scipy.misc import derivative
from scipy.special import expit as sigmoid
from scipy.special import logit

def func(ti, yr, yc, si):
    qi = sigmoid(si)
    if ti == 1:
        return - yr*np.log(qi/(1-qi)) - yc*np.log(1-qi)
    else:
        return yr*np.log(qi/(1-qi)) + yc*np.log(1-qi)

    
def derivative_values(ti, yr, yc, si):
    def func_fixed_y(si_val):
        return func(ti, yr, yc, si_val)
    deriv = derivative(func_fixed_y, si, dx=1e-6)
    return deriv


def bst(ti, yr, yc):
    epsilon = 1e-5
    # corner case
    if yr == 0:
        return epsilon
    elif yc == 0:
        return 1 - epsilon
    
    roi_l = 0.0
    roi_r = 1.0
    roi_star = (roi_l + roi_r) / 2
    

    while np.abs(roi_r - roi_l) > epsilon:
        derivatives_at_si = derivative_values(ti, yr, yc, logit(roi_star))
        if np.abs(derivatives_at_si) < epsilon:
            break
        if derivatives_at_si > 0:
            roi_r = roi_star
        else:
            roi_l = roi_star
        roi_star = (roi_l + roi_r) / 2
    return roi_star


results = np.zeros(T_cali.shape)

# iterate
for i in range(T_cali.shape[0]):
    # call bst
    results[i] = bst(T_cali[i], Y_conv_cali[i], Y_visit_cali[i])

/tmp/ipykernel_1526/1185985485.py:17: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  deriv = derivative(func_fixed_y, si, dx=1e-6)


In [6]:
delta_roi = np.abs(results - roi_rank_pre_cali)

In [7]:
np.unique(delta_roi)

array([2.90425962e-07, 1.12657355e-06, 1.28193162e-06, ...,
       9.99985589e-01, 9.99985828e-01, 9.99986185e-01])

### (iii) Infer DRP's MC Dropout model to obtain $\hat{r}(x)$.

In [8]:
%%time
import sklearn 
import sklearn.metrics
from metric.Metric import *
from concurrent.futures import ProcessPoolExecutor

DRP_aucc_cali_mc_list = []

for i in range(count):
    
    print("iteration = ", i + 1)
    
    final_model = get_roi_rank_model_with_dropout()
    final_model.load_weights('../model_file/roi/criteo/final_model/roi_rank/roi_rank_criteo_model_{}_{}.h5'.format(i+1, sample))

    # multiple prediction
    n_iterations = 10
    all_predictions_cali = []
    
    for j in range(n_iterations):
        print(j)
        
        print("cali")
        predictions = final_model.predict([X_cali, T_cali, Y_conv_cali, Y_visit_cali])
        all_predictions_cali.append(predictions)
        DRP_aucc = get_uplift_model_aucc_no_show(t=(T_cali > 0.5).flatten(), y_reward=Y_conv_cali.flatten(), y_cost=Y_visit_cali.flatten(), roi_pred=predictions.flatten(), quantile=200)
        DRP_aucc_cali_list.append(DRP_aucc[0])
    
    print("\n")
    
# obtain mean and std
mean_pred = np.mean(all_predictions_cali, axis=0)
std_pred = np.std(all_predictions_cali, axis=0)


iteration =  1
0
cali
4557/4557 [==============================] - 12s 3ms/step
AUCC =  0.7169969966253281
1
cali
4557/4557 [==============================] - 11s 2ms/step
AUCC =  0.721880202053059
2
cali
4557/4557 [==============================] - 12s 3ms/step
AUCC =  0.7185034065182082
3
cali
4557/4557 [==============================] - 11s 3ms/step
AUCC =  0.716480580521801
4
cali
4557/4557 [==============================] - 11s 2ms/step
AUCC =  0.7182949566115989
5
cali
4557/4557 [==============================] - 12s 3ms/step
AUCC =  0.720145596486794
6
cali
4557/4557 [==============================] - 11s 2ms/step
AUCC =  0.72054982357939
7
cali
4557/4557 [==============================] - 12s 3ms/step
AUCC =  0.7201097031818404
8
cali
4557/4557 [==============================] - 12s 3ms/step
AUCC =  0.7188971840201609
9
cali
4557/4557 [==============================] - 11s 2ms/step
AUCC =  0.7198594823118163


CPU times: user 2min 53s, sys: 10.3 s, total: 3min 3s
Wall time: 2mi

### (iv) Calculate $\hat{q}$ by Algorithm 3

In [29]:
# Get scores
cal_scores = delta_roi/(std_pred + 1e-5)
n = len(cal_scores)

# set the error rate
alpha = 0.30
# Get the score quantile
qhat = np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n, interpolation='higher')

### (v) Select $\widetilde{roi}$'s equation form from 5a to 5c.

In [116]:
# For mc only


ROI_1 = (4.666666*roi_rank_pre_cali + std_pred) # AUCC 
ROI_2 = roi_rank_pre_cali/(std_pred + 1e5)
ROI_3 = 0.95*roi_rank_pre_cali + std_pred
DRP_aucc_cali = get_uplift_model_aucc_no_show(t=(T_cali > 0.5).flatten(), y_reward=Y_conv_cali.flatten(), y_cost=Y_visit_cali.flatten(), roi_pred=ROI_3.flatten(), quantile=200)

AUCC =  0.7209406214095225


In [98]:
ROI_1 = roi_rank_pre_cali*(113.95*roi_rank_pre_cali + qhat*std_pred) # AUCC 
ROI_2 = roi_rank_pre_cali/(qhat*std_pred + 1e5)
ROI_3 = (227.45*roi_rank_pre_cali + qhat*std_pred) #
DRP_aucc_cali = get_uplift_model_aucc_no_show(t=(T_cali > 0.5).flatten(), y_reward=Y_conv_cali.flatten(), y_cost=Y_visit_cali.flatten(), roi_pred=ROI_3.flatten(), quantile=200)

AUCC =  0.7213974362685069


# Third, on the test set:

### (i) Infer DRP model to obtain $\hat{roi}$.

In [10]:
import sklearn 
import sklearn.metrics
from metric.Metric import *

count = 1
sample = 1.0

DRP_aucc_test_list = []
roi_rank_pre_test_list = []
for i in range(count):
    
    print("iteration = ", i + 1)
    
    final_model = get_roi_rank_model()
    final_model.load_weights('../model_file/roi/criteo/final_model/roi_rank/roi_rank_criteo_model_{}_{}.h5'.format(i+1, sample))

    roi_rank_pre_test = final_model.predict([X_test, T_test, Y_conv_test, Y_visit_test])
    roi_rank_pre_test_list.append(roi_rank_pre_test)
    DRP_aucc = get_uplift_model_aucc_no_show(t=(T_test > 0.5).flatten(), y_reward=Y_conv_test.flatten(), y_cost=Y_visit_test.flatten(), roi_pred=roi_rank_pre_test.flatten(), quantile=200)
    DRP_aucc_test_list.append(DRP_aucc)
    
    print("\n")

iteration =  1
22785/22785 [==============================] - 47s 2ms/step
AUCC =  0.7223578876322072




In [11]:
# store test aucc for pic 
import pandas as pd

def get_aucc_cost_curve(aucc_list):
    delta_cost_list_group = np.array([aucc[1] for aucc in aucc_list])
    delta_reward_list_group = np.array([aucc[2] for aucc in aucc_list])
    
    avg_delta_cost_list = np.mean(delta_cost_list_group, axis=0)
    avg_delta_reward_list = np.mean(delta_reward_list_group, axis=0)
    
    df_aucc_cost_curve = pd.DataFrame(avg_delta_cost_list, columns=['delta_cost'])
    df_aucc_cost_curve['delta_reward'] = avg_delta_reward_list
    
    return df_aucc_cost_curve

DRP_avg_aucc_cost_curve = get_aucc_cost_curve(DRP_aucc_test_list)
print("aucc = ", np.sum(DRP_avg_aucc_cost_curve['delta_reward'].values) / (DRP_avg_aucc_cost_curve['delta_reward'].values[-1] * 201))
DRP_avg_aucc_cost_curve.to_csv("../figure/mt/a_DRP_avg_aucc_cost_curve_1.0.csv")

aucc =  0.7223578876322072


### (ii) Infer DRP's MC Dropout to obtain $\hat{r}(x)$.

In [12]:
%%time
import time
import sklearn 
import sklearn.metrics
from metric.Metric import *
from concurrent.futures import ProcessPoolExecutor
sys.path.append("..")
from model.uplift_model import *
from model.roi_model import *


count = 1
sample = 1.0

DRP_aucc_test_mc_list = []

for i in range(count):
    
    print("iteration = ", i + 1)
    
    final_model = get_roi_rank_model_with_dropout()
    final_model.load_weights('../model_file/roi/criteo/final_model/roi_rank/roi_rank_criteo_model_{}_{}.h5'.format(i+1, sample))

    # multiple prediction
    n_iterations = 10
    all_predictions_test = []
    
    for j in range(n_iterations):
        print(j)
        
        print("test")
        predictions = final_model.predict([X_test, T_test, Y_conv_test, Y_visit_test])
        all_predictions_test.append(predictions)
        DRP_aucc_mc = get_uplift_model_aucc_no_show(t=(T_test > 0.5).flatten(), y_reward=Y_conv_test.flatten(), y_cost=Y_visit_test.flatten(), roi_pred=predictions.flatten(), quantile=200)
        DRP_aucc_test_mc_list.append(DRP_aucc_mc[0])
    
    print("\n")
    
# cal std and mean
mean_pred_test = np.mean(all_predictions_test, axis=0)
std_pred_test = np.std(all_predictions_test, axis=0)

end_time = time.time()
print(end_time)

iteration =  1
0
test
22785/22785 [==============================] - 55s 2ms/step
AUCC =  0.7211992024301702
1
test
22785/22785 [==============================] - 60s 3ms/step
AUCC =  0.719815928680756
2
test
22785/22785 [==============================] - 60s 3ms/step
AUCC =  0.7212853003230847
3
test
22785/22785 [==============================] - 61s 3ms/step
AUCC =  0.7212452931140114
4
test
22785/22785 [==============================] - 55s 2ms/step
AUCC =  0.7198411037100763
5
test
22785/22785 [==============================] - 58s 3ms/step
AUCC =  0.7211516023028189
6
test
22785/22785 [==============================] - 56s 2ms/step
AUCC =  0.7211653440804371
7
test
22785/22785 [==============================] - 58s 3ms/step
AUCC =  0.7214829483225055
8
test
22785/22785 [==============================] - 56s 2ms/step
AUCC =  0.7208709349544241
9
test
22785/22785 [==============================] - 56s 2ms/step
AUCC =  0.7209984000372961


1708415573.3238635
CPU times: user 14min 30s

### (iii) With $\hat{q}$  and the selected expression, $\widetilde{roi}(x_{test})$ is obtained.

In [13]:
# # calculate DRP-MC aucc
rDRP_aucc_test_list = []
ROI_1 = (0.95*roi_rank_pre_test + std_pred_test) # 
rDRP_aucc = get_uplift_model_aucc_no_show(t=(T_test > 0.5).flatten(), y_reward=Y_conv_test.flatten(), y_cost=Y_visit_test.flatten(), roi_pred=ROI_1.flatten(), quantile=200)
rDRP_aucc_test_list.append(rDRP_aucc)

# save DRP-MC AUCC result

rDRP_avg_aucc_cost_curve = get_aucc_cost_curve(rDRP_aucc_test_list)
print("aucc = ", np.sum(rDRP_avg_aucc_cost_curve['delta_reward'].values) / (rDRP_avg_aucc_cost_curve['delta_reward'].values[-1] * 201))
rDRP_avg_aucc_cost_curve.to_csv("../figure/mt/a_DRP_MC_avg_aucc_cost_curve_1.0.csv")


AUCC =  0.7253356268240889
aucc =  0.7253356268240889


In [14]:
# calculate rDRP aucc
rDRP_aucc_test_list = []
ROI_1 = (227.45*roi_rank_pre_test + qhat*std_pred_test)# 
rDRP_aucc = get_uplift_model_aucc_no_show(t=(T_test > 0.5).flatten(), y_reward=Y_conv_test.flatten(), y_cost=Y_visit_test.flatten(), roi_pred=ROI_1.flatten(), quantile=200)
rDRP_aucc_test_list.append(rDRP_aucc)

# save rDRP AUCC result

rDRP_avg_aucc_cost_curve = get_aucc_cost_curve(rDRP_aucc_test_list)
print("aucc = ", np.sum(rDRP_avg_aucc_cost_curve['delta_reward'].values) / (rDRP_avg_aucc_cost_curve['delta_reward'].values[-1] * 201))
rDRP_avg_aucc_cost_curve.to_csv("../figure/mt/rDRP_avg_aucc_cost_curve_1.0.csv")


AUCC =  0.7290035664594612
aucc =  0.7290035664594612
